In [1]:
# lets us run the notebook from within our django project
import django_initializer

# Simple-Salesforce
## PCP REST API
This is a binder outlining how we can query data in the patient connections platform using rest APIs in order to analyze/outline business processes

## Establishing a connection to the server

This first code block is going to walk through how we can gain access to the data within salesforce to start writing queries. First you will need an account for the salesforce platform that you are trying to access. 

After you get your account set up, you should store your credentials securely at the operating system level. We'll import the operating system and then we can import a python package that will let us control salesforce programatically

In [10]:
# ------------------------------- #
# imports 
# ------------------------------- #

import os
from simple_salesforce import Salesforce

# defining our log in credentials, saved as environment variables
username = os.environ.get('FORCE_PROD_USER')
password = os.environ.get('FORCE_PROD_PW')
security_token = os.environ.get('FORCE_PROD_TOKEN')


# ------------------------------- #
# imports 
# ------------------------------- #

def connect_to_prod(use_proxies):
    """ proxies give us elevated security during our connections sessions """
    if use_proxies:
        proxies = {
            'http': 'http://foo:bar@us_proxy_indy.xh1.lilly.com:9000',
            'https': 'http://foo:bar@us_proxy_indy.xh1.lilly.com:9000'
        }

        sf = Salesforce(username=username, password=password, security_token=security_token, proxies=proxies)
    else:
        sf = Salesforce(username=username, password=password, security_token=security_token)
    return sf

# call the connection to see if we can access data
connect_to_prod(True)

## Setting up functions

I wrote the connection into salesforce as a function, this way it is reusable. This specific function calls a connection into our production env.. kinda risky... but don't do anything stupid and you'll be fine, we're just repoting on data not manipulating it yet.

To perform salesforce development tasks, we should really use a connected app architecture and connect into a development environment. This connected app architecture is built through Django... and is a lot more... advanced...

For now I am just going to show off how to query data in order to write processes and conduct reports. I am going to make another reusable function so that writing queries is easier in the future. We can call the function by passing in our salesforce connection and providing a "SOQL" statement. SOQL is salesforce's modified version of SQL that is built specifically for the FORCE.com platform.


In [11]:
def query(sf, soql):
    # uses query_all so results are not paginated...
    soql_query = sf.query_all(soql)
    
    size = soql_query['totalSize']
    done = soql_query['done']
    # records are the actual data returned inside of the function's json
    records = soql_query['records']
    """ i am reformatting the return result so that this custom function just gives a clean python list """
    returned_records = []
    for record in records:
        record_fields = {}
        attributes = record['attributes']
        for key in record.keys():
            if record[key] != attributes:
                record_fields[key] = record[key]
        returned_records.append(record_fields)
    # cleaned python list of records, each record will have JSON to hold data
    return returned_records

## List queries

this support function will allow us to do more interesting queries... you'll see it in action shortly, but it will be allow us to write queries that take in a list of specific values and return data connected to that specific list of records. For example, if I want to look at 10 specific records and return related record data specific to only those 10 records, we would call this function.

In [12]:
def query_list(sf, soql, the_list):
    """ when using the IN operator, if the list contains more than 20k characters, it hits a limit so i'll partition the query using chunks of length 700 """
    records = []
    number_of_lists = round(len(the_list) / 700)
    if number_of_lists == 0:
        number_of_lists = 1
    start_index = 0
    try:
        partition_length = round(len(the_list) / number_of_lists)
    except:
        partition_length = round(len(the_list))
    end_index = partition_length
    for i in range(number_of_lists):
        sub_list = the_list[start_index:end_index]
        list_string = format_list(sub_list)
        sub_soql = soql + "%s" % list_string
        try:
            sf_data = query(sf, sub_soql)
        except ValueError:
            sf_data = []
            print("SOQL error, no data returned")
        for record in sf_data:
            records.append(record)
        start_index = end_index
        end_index = partition_length * (i + 2)
    # returning all the records found in the query
    return records

## The PCP program object

The salesforce application that we are connected to is called the 'Patient Connections Platform'. This app is built off of a series of totally custom objects and workflows. The core object of the app is called the 'Program'.

A program record represents a single patient's enrollment into a Customer Support Program. I've created a bunch of utility functions that will outline the data model for a program and allow us to quickly query records in the PCP. Reading through this will help you understand the related objects to a CSP patient's program.

*remember, these functions are designed to abstract away complexity, i am showing off the usage of the simple-salesforce package through a notebook but structuring these functions through a file directory will allow you to make them reusable*

In [13]:
# --------------------------------- #
# consent parsing
# --------------------------------- #

def parse_consents(program):
    """ consents are processed upon patient enrollment into a CSP and stored to either a patient/caregiver through
     the program record """
    try:
        consents = program['Consent_Auth_Preferences__r']['records']
        consent_data = []
        for record in consents:
            consent_record = {}
            for key in record:
                if key != 'attributes':
                    consent_record[key] = record[key]
            consent_data.append(consent_record)
    except KeyError:
        consent_data = []

    # return a list of all the consent records cleaned up
    return consent_data


# --------------------------------- #
# coverage parsing
# --------------------------------- #

def parse_coverages(program):
    """ takes in a program and returns a list containing a list of copay coverages and a list of insurance coverages """
    try:
        # store copay records in first list, insurance records in the second list
        coverage_data = [[], []]
        coverages = program['Coverage1__r']['records']
        for record in coverages:
            try:
                if record['RecordTypeId'] == '012f20000001HL3AAM':
                    copay_coverage_record = {}
                    for key in record:
                        if key != 'attributes' and key != 'RecordTypeId':
                            copay_coverage_record[key] = record[key]
                    coverage_data[0].append(copay_coverage_record)

                elif record['RecordTypeId'] == '012f20000001HL4AAM':
                    insurance_coverage_record = {}
                    for key in record:
                        if key != 'attributes' and key != 'RecordTypeId':
                            insurance_coverage_record[key] = record[key]
                    coverage_data[1].append(insurance_coverage_record)

            except KeyError:
                print("queries that incorporate coverage records must include the RecordTypeId field")

    # return an empty data model if we pass the method an invalid data structure
    except KeyError:
        coverage_data = [[], []]
    return coverage_data


def filter_active_coverages(coverages):
    """ takes a list of coverage records and parses out any inactive records (good for separating ins and copay) """
    active_coverages = []
    for coverage in coverages:
        try:
            if coverage['DTPC_Record_Status__c'] == 'Active':
                active_coverages.append(coverage)
        except KeyError:
            print("this method requires you to pull the DTPC_Record_Status__c from coverage")
    return active_coverages


def parse_old_coverages(program):
    try:
        # store copay records in first list, insurance records in the second list
        coverage_data = []
        coverages = program['Coverage__r']['records']
        for record in coverages:
            old_coverage_record = {}
            for key in record:
                if key != 'attributes':
                    old_coverage_record[key] = record[key]
            coverage_data.append(old_coverage_record)
    # return an empty data model if we pass the method an invalid data structure
    except KeyError:
        coverage_data = []
    return coverage_data


def filter_active_old_coverages(coverages):
    active_coverages = []
    for coverage in coverages:
        try:
            if coverage['Coverage_Status__c'] == 'Active':
                active_coverages.append(coverage)
        except KeyError:
            print("this method requires your query to pull the Coverage_Status__c field from coverage")
    return active_coverages


# --------------------------------- #
# claims parsing
# --------------------------------- #

def parse_claims(program):
    try:
        claims = program['Claims_Adjudication__r']['records']
        claims_data = []
        for record in claims:
            claim_record = {}
            for key in record:
                if key !='attributes':
                    claim_record[key] = record[key]
            claims_data.append(claim_record)
    except KeyError:
        claims_data = []

    return claims_data


# --------------------------------- #
# dispense parsing
# --------------------------------- #

def parse_dispenses(program):
    try:
        dispenses = program['Specialty_Pharmacy_Dispenses__r']['records']
        dispense_data = []
        for record in dispenses:
            dispense_record = {}
            for key in record:
                if key != 'attributes':
                    dispense_record[key] = record[key]
            dispense_data.append(dispense_record)
    except KeyError:
        dispense_data = []

    return dispense_data


# --------------------------------- #
# program services parsing
# --------------------------------- #

def parse_program_services(program):
    try:
        services = program['Program_Services__r']['records']
        services_data = []
        for record in services:
            service_record = {}
            for key in record:
                if key != 'attributes':
                    service_record[key] = record[key]
            services_data.append(service_record)
    except KeyError:
        services_data = []

    return services_data


# --------------------------------- #
# agent workflow parsing
# --------------------------------- #

def parse_cases(program):
    try:
        cases = program['Cases__r']['records']
        case_data = []
        for record in cases:
            case_record = {}
            for key in record:
                if key != 'attributes':
                    case_record[key] = record[key]
            case_data.append(case_record)
    except KeyError:
        case_data = []

    return case_data


def parse_document_logs(program):
    logs = program['Document_Logs__r']
    try:
        document_logs = logs['records']
        document_data = []
        for record in document_logs:
            document_record = {}
            for key in record:
                if key != 'attributes':
                    document_record[key] = record[key]
            document_data.append(document_record)

    except KeyError:
        document_data = []

    # return the two separate data structures
    return document_data

## Process based reporting

We are now going to use all of the functions that I just shared to show off how we can construct different types of reports that access data in the PCP. This specific notebook will monitor the inbound records recieved via IQVIA for the SP Portal.

## SP Portal

This process is going to analyze data that was loaded into salesforce from IQVIA, the data is manually processed by call center agents today... we are working on automating that...

In [22]:
# first, establish a connetion into the platform
sf = connect_to_prod(True)

# now write a SOQL query to define the data we want to analyze
soql = "SELECT Name, Fax_Category__c, Fax_Status__c, CreatedDate, Smartform_Registration__r.Hub_Patient_ID__c, " \
           "Smartform_Registration__r.PA_Submitted__c, Smartform_Registration__r.PA_Submitted_Date__c, " \
           "Smartform_Registration__r.PA_Outcome__c, Smartform_Registration__r.PA_Denial_Reason__c, " \
           "Smartform_Registration__r.PA_Expired__c, Smartform_Registration__r.Appeal_Submitted__c, " \
           "Smartform_Registration__r.Appeal_Submitted_Date__c, Smartform_Registration__r.Appeal_Outcome__c, " \
           "Smartform_Registration__r.Appeal_Denial_Reason__c, Smartform_Registration__r.Benefit_Converted__c, " \
           "Smartform_Registration__r.Date_Action_Recorded__c, " \
           "Smartform_Registration__r.DTPC_Prescriber_First_Name__c, " \
           "Smartform_Registration__r.DTPC_Prescriber_Last_Name__c, " \
           "Smartform_Registration__r.DTPC_Prescriber_Address__c, Smartform_Registration__r.Prescriber_Address_2__c, " \
           "Smartform_Registration__r.DTPC_Prescriber_City__c, Smartform_Registration__r.DTPC_Prescriber_State__c, " \
           "Smartform_Registration__r.Prescriber_Zip__c, Smartform_Registration__r.DTPC_Prescriber_Phone__c, " \
           "Smartform_Registration__r.NPI__c, Smartform_Registration__r.DTPC_Copay_Card_Number__c, " \
           "Smartform_Registration__r.Insurance_Effective_Date__c, Smartform_Registration__r.DTPC_DocuSign_RxBIN__c, " \
           "Smartform_Registration__r.DTPC_DocuSign_RxPCN__c, Smartform_Registration__r.NCPDP__c, " \
           "Smartform_Registration__r.DTPC_DocuSign_RxGroup__c, " \
           "Smartform_Registration__r.DTPC_Primary_Insurance_Number__c,	" \
           "Smartform_Registration__r.DTPC_Primary_Insurance_Company_Phone__c, " \
           "Smartform_Registration__r.DTPC_Primary_Insurance_Company__c, " \
           "Smartform_Registration__r.DTPC_Insurance_Plan_Name__c, Smartform_Registration__r.Pharmacy_Name__c " \
           "FROM DTPC_Document__c " \
           "WHERE RecordTypeId = '012f2000000QLPSAA4' AND Fax_Status__c = 'New'"

# return the results of the query in a list using our custom function
docs = query(sf, soql)

# print out the number of docs our query returned
print(len(docs))

824


In [23]:
def group_documents_by_copay_id(documents):
    copay_card_map = {}
    for document in documents:
        # sorting out the data from related registration, fetching copay card id
        registration_data = document['Smartform_Registration__r']
        copay_card_id = registration_data['DTPC_Copay_Card_Number__c']
        # store each registration according to the copay card id
        if copay_card_id in copay_card_map.keys():
            copay_card_map[copay_card_id][0].append(registration_data)
            copay_card_map[copay_card_id][1].append(document)
        else:
            copay_card_map[copay_card_id] = [[registration_data], [document]]
    # return the map
    return copay_card_map

## Evaluate New Inbound Copay Cards
*Simple function will print out all of the card ids that have recieved a new update*

In [26]:
new_copay_cards_recieved = group_documents_by_copay_id(docs)
def parse_copay_map(new_copay_cards_recieved):
    for card in new_copay_cards_recieved:
        print(card)

parse_copay_map(new_copay_cards_recieved)

E46103092427
E46100669237
E49101049192
E49102582267
E46101972426
E46103084797
E46102953615
E46103128688
E46102756001
E46101323945
E46103056446
E49102000914
E46103056167
E49102662763
E49102665282
E49102665421
E49102661080
E49102669484
E49100655378
E46103120070
E49102670394
E49102586041
E49101450646
E46101555151
E46103069395
E49102641346
E49102640221
E46102996876
E46102047111
E49102623072
E46101339273
E49102670533
E49102657586
E49102672560
E49102597102
E46103136598
E46103017806
E46103037475
E46103062608
E46102386401
E49102473273
E46102981194
E46102860376
E49102573306
E49100607703
E46103082482
E49102538162
E46100898203
E46101264447
E49102659268
E46101552148
E49102673266
E46102541033
E46103125397
E46103116715
E46102379402
E46103116157
E46103118956
E49101393035
E49102635323
E49102616426
E46102854986
E49100689110
E49102588980
E46101383517
E46102933807
E46103134217
E46103124625
E46103124904
E49102659752
E49102666825
E46101620327
E49102659407
E49102649257
E46103093264
E49102616631
E49102092406

## Data Cleaning
*parse the new card values for update types*

In [43]:
def create_update_buckets(copay_card_map):
    """ creating the different buckets for updating related program records """
    refined_map = {}
    for copay_card in copay_card_map:
        # fetching all data received for this card
        registration_list = copay_card_map[copay_card][0]
        document_list = copay_card_map[copay_card][1]

# -------------------------------------------------- #
# instantiating the different types of updates
# -------------------------------------------------- #

        pa_bucket = {
            'Date_Action_Recorded__c': '', 'PA_Submitted__c': '', 'PA_Submitted_Date__c': '', 'PA_Outcome__c': '',
            'PA_Denial_Reason__c': '', 'PA_Expired__c': ''
        }

        appeal_bucket = {
            'Date_Action_Recorded__c': '', 'Appeal_Submitted__c': '', 'Appeal_Submitted_Date__c': '',
            'Appeal_Outcome__c': '', 'Appeal_Denial_Reason__c': ''
        }

        benefit_conversions = []

        prescriber_bucket = {
            'Date_Action_Recorded__c': '', 'DTPC_Prescriber_First_Name__c': '', 'DTPC_Prescriber_Last_Name__c': '',
            'DTPC_Prescriber_Address__c': '',
            'Prescriber_Address_2__c': '', 'DTPC_Prescriber_City__c': '',
            'DTPC_Prescriber_State__c': '', 'Prescriber_Zip__c': '', 'DTPC_Prescriber_Phone__c': '', 'NPI__c': ''
        }

        insurance_bucket = {
            'Date_Action_Recorded__c': '', 'Insurance_Effective_Date__c': '', 'DTPC_DocuSign_RxBIN__c': '',
            'DTPC_DocuSign_RxPCN__c': '', 'NCPDP__c': '', 'DTPC_DocuSign_RxGroup__c': '',
            'DTPC_Primary_Insurance_Number__c': '', 'DTPC_Primary_Insurance_Company_Phone__c': '',
            'DTPC_Primary_Insurance_Company__c': '', 'DTPC_Insurance_Plan_Name__c': ''
        }

        pharmacy_name = ['', '']

        for registration in registration_list:

            # establishing the date recorded on the status update record
            update_date = str(registration['Date_Action_Recorded__c'])
            registration_program_id = registration['Hub_Patient_ID__c']

            """ the update buckets will be filled in with data from the most recent date of action if multiple actions
             occur on the same field """

# -------------------------------------------------- #
# filling in the bucket for prior_authorizations
# -------------------------------------------------- #

            if registration['PA_Submitted__c'] is not None:
                if update_date >= pa_bucket['Date_Action_Recorded__c']:
                    pa_bucket['Date_Action_Recorded__c'] = update_date
                    pa_bucket['PA_Submitted__c'] = registration['PA_Submitted__c']

            if registration['PA_Submitted_Date__c'] is not None:
                if update_date >= pa_bucket['Date_Action_Recorded__c']:
                    pa_bucket['Date_Action_Recorded__c'] = update_date
                    pa_bucket['PA_Submitted_Date__c'] = registration['PA_Submitted_Date__c']

            if registration['PA_Outcome__c'] is not None:
                if update_date >= pa_bucket['Date_Action_Recorded__c']:
                    pa_bucket['Date_Action_Recorded__c'] = update_date
                    pa_bucket['PA_Outcome__c'] = registration['PA_Outcome__c']

            if registration['PA_Denial_Reason__c'] is not None:
                if update_date >= pa_bucket['Date_Action_Recorded__c']:
                    pa_bucket['Date_Action_Recorded__c'] = update_date
                    pa_bucket['PA_Denial_Reason__c'] = registration['PA_Denial_Reason__c']

            if registration['PA_Expired__c'] is not None:
                if update_date >= pa_bucket['Date_Action_Recorded__c']:
                    pa_bucket['Date_Action_Recorded__c'] = update_date
                    pa_bucket['PA_Expired__c'] = registration['PA_Expired__c']


# -------------------------------------------------- #
# filling in the bucket for prior_authorizations
# -------------------------------------------------- #

            if registration['Appeal_Submitted__c'] is not None:
                if update_date >= appeal_bucket['Date_Action_Recorded__c']:
                    appeal_bucket['Date_Action_Recorded__c'] = update_date
                    appeal_bucket['Appeal_Submitted__c'] = registration['Appeal_Submitted__c']

            if registration['Appeal_Submitted_Date__c'] is not None:
                if update_date >= appeal_bucket['Date_Action_Recorded__c']:
                    appeal_bucket['Date_Action_Recorded__c'] = update_date
                    appeal_bucket['Appeal_Submitted_Date__c'] = registration['Appeal_Submitted_Date__c']

            if registration['Appeal_Outcome__c'] is not None:
                if update_date >= appeal_bucket['Date_Action_Recorded__c']:
                    appeal_bucket['Date_Action_Recorded__c'] = update_date
                    appeal_bucket['Appeal_Outcome__c'] = registration['Appeal_Outcome__c']

            if registration['Appeal_Denial_Reason__c'] is not None:
                if update_date >= appeal_bucket['Date_Action_Recorded__c']:
                    appeal_bucket['Date_Action_Recorded__c'] = update_date
                    appeal_bucket['Appeal_Denial_Reason__c'] = registration['Appeal_Denial_Reason__c']

            # --------------------------------------------------------------------------------------------------------
            # filling out the benefit conversions
            # --------------------------------------------------------------------------------------------------------
            if registration['Benefit_Converted__c'] is not None:
                benefit_conversions.append((registration['Benefit_Converted__c'], update_date))

            # --------------------------------------------------------------------------------------------------------
            # filling out the prescriber information
            # --------------------------------------------------------------------------------------------------------
            """ take the prescriber information from the most recent document """
            if registration['DTPC_Prescriber_First_Name__c'] != prescriber_bucket['DTPC_Prescriber_First_Name__c']:
                if update_date >= prescriber_bucket['Date_Action_Recorded__c']:
                    prescriber_bucket['Date_Action_Recorded__c'] = update_date
                    prescriber_bucket['DTPC_Prescriber_First_Name__c'] = registration['DTPC_Prescriber_First_Name__c']

            if registration['DTPC_Prescriber_Last_Name__c'] != prescriber_bucket['DTPC_Prescriber_Last_Name__c']:
                if update_date >= prescriber_bucket['Date_Action_Recorded__c']:
                    prescriber_bucket['Date_Action_Recorded__c'] = update_date
                    prescriber_bucket['DTPC_Prescriber_Last_Name__c'] = registration['DTPC_Prescriber_Last_Name__c']

            if registration['DTPC_Prescriber_Address__c'] != prescriber_bucket['DTPC_Prescriber_Address__c']:
                if update_date >= prescriber_bucket['Date_Action_Recorded__c']:
                    prescriber_bucket['Date_Action_Recorded__c'] = update_date
                    prescriber_bucket['DTPC_Prescriber_Address__c'] = registration['DTPC_Prescriber_Address__c']

            if registration['Prescriber_Address_2__c'] != prescriber_bucket['Prescriber_Address_2__c']:
                if update_date >= prescriber_bucket['Date_Action_Recorded__c']:
                    prescriber_bucket['Date_Action_Recorded__c'] = update_date
                    prescriber_bucket['Prescriber_Address_2__c'] = registration['Prescriber_Address_2__c']

            if registration['DTPC_Prescriber_City__c'] != prescriber_bucket['DTPC_Prescriber_City__c']:
                if update_date >= prescriber_bucket['Date_Action_Recorded__c']:
                    prescriber_bucket['Date_Action_Recorded__c'] = update_date
                    prescriber_bucket['DTPC_Prescriber_City__c'] = registration['DTPC_Prescriber_City__c']

            if registration['DTPC_Prescriber_State__c'] != prescriber_bucket['DTPC_Prescriber_State__c']:
                if update_date >= prescriber_bucket['Date_Action_Recorded__c']:
                    prescriber_bucket['Date_Action_Recorded__c'] = update_date
                    prescriber_bucket['DTPC_Prescriber_State__c'] = registration['DTPC_Prescriber_State__c']

            if registration['Prescriber_Zip__c'] != prescriber_bucket['Prescriber_Zip__c']:
                if update_date >= prescriber_bucket['Date_Action_Recorded__c']:
                    prescriber_bucket['Date_Action_Recorded__c'] = update_date
                    prescriber_bucket['Prescriber_Zip__c'] = registration['Prescriber_Zip__c']

            if registration['DTPC_Prescriber_Phone__c'] != prescriber_bucket['DTPC_Prescriber_Phone__c']:
                if update_date >= prescriber_bucket['Date_Action_Recorded__c']:
                    prescriber_bucket['Date_Action_Recorded__c'] = update_date
                    prescriber_bucket['DTPC_Prescriber_Phone__c'] = registration['DTPC_Prescriber_Phone__c']

            if registration['NPI__c'] != prescriber_bucket['NPI__c']:
                if update_date >= prescriber_bucket['Date_Action_Recorded__c']:
                    prescriber_bucket['Date_Action_Recorded__c'] = update_date
                    prescriber_bucket['NPI__c'] = registration['NPI__c']

            # --------------------------------------------------------------------------------------------------------
            # filling out the insurance information
            # --------------------------------------------------------------------------------------------------------
            """ take the insurance information from the most recent update """
            if registration['Insurance_Effective_Date__c'] != insurance_bucket['Insurance_Effective_Date__c']:
                if update_date >= insurance_bucket['Date_Action_Recorded__c']:
                    insurance_bucket['Date_Action_Recorded__c'] = update_date
                    insurance_bucket['Insurance_Effective_Date__c'] = registration['Insurance_Effective_Date__c']

            if registration['DTPC_DocuSign_RxBIN__c'] != insurance_bucket['DTPC_DocuSign_RxBIN__c']:
                if update_date >= insurance_bucket['Date_Action_Recorded__c']:
                    insurance_bucket['Date_Action_Recorded__c'] = update_date
                    insurance_bucket['DTPC_DocuSign_RxBIN__c'] = registration['DTPC_DocuSign_RxBIN__c']

            if registration['DTPC_DocuSign_RxPCN__c'] != insurance_bucket['DTPC_DocuSign_RxPCN__c']:
                if update_date >= insurance_bucket['Date_Action_Recorded__c']:
                    insurance_bucket['Date_Action_Recorded__c'] = update_date
                    insurance_bucket['DTPC_DocuSign_RxPCN__c'] = registration['DTPC_DocuSign_RxPCN__c']

            if registration['NCPDP__c'] != insurance_bucket['NCPDP__c']:
                if update_date >= insurance_bucket['Date_Action_Recorded__c']:
                    insurance_bucket['Date_Action_Recorded__c'] = update_date
                    insurance_bucket['NCPDP__c'] = registration['NCPDP__c']

            if registration['DTPC_DocuSign_RxGroup__c'] != insurance_bucket['DTPC_DocuSign_RxGroup__c']:
                if update_date >= insurance_bucket['Date_Action_Recorded__c']:
                    insurance_bucket['Date_Action_Recorded__c'] = update_date
                    insurance_bucket['DTPC_DocuSign_RxGroup__c'] = registration['DTPC_DocuSign_RxGroup__c']

            if registration['DTPC_Primary_Insurance_Number__c'] != insurance_bucket['DTPC_Primary_Insurance_Number__c']:
                if update_date >= insurance_bucket['Date_Action_Recorded__c']:
                    insurance_bucket['Date_Action_Recorded__c'] = update_date
                    insurance_bucket['DTPC_Primary_Insurance_Number__c'] = \
                        registration['DTPC_Primary_Insurance_Number__c']

            if registration['DTPC_Primary_Insurance_Company_Phone__c'] != \
                    insurance_bucket['DTPC_Primary_Insurance_Company_Phone__c']:
                if update_date >= insurance_bucket['Date_Action_Recorded__c']:
                    insurance_bucket['Date_Action_Recorded__c'] = update_date
                    insurance_bucket['DTPC_Primary_Insurance_Company_Phone__c'] = \
                        registration['DTPC_Primary_Insurance_Company_Phone__c']

            if registration['DTPC_Primary_Insurance_Company__c'] != \
                    insurance_bucket['DTPC_Primary_Insurance_Company__c']:
                if update_date >= insurance_bucket['Date_Action_Recorded__c']:
                    insurance_bucket['Date_Action_Recorded__c'] = update_date
                    insurance_bucket['DTPC_Primary_Insurance_Company__c'] = \
                        registration['DTPC_Primary_Insurance_Company__c']

            if registration['DTPC_Insurance_Plan_Name__c'] != insurance_bucket['DTPC_Insurance_Plan_Name__c']:
                if update_date >= insurance_bucket['Date_Action_Recorded__c']:
                    insurance_bucket['Date_Action_Recorded__c'] = update_date
                    insurance_bucket['DTPC_Insurance_Plan_Name__c'] = registration['DTPC_Insurance_Plan_Name__c']

            # --------------------------------------------------------------------------------------------------------
            # filling out the pharmacy location
            # --------------------------------------------------------------------------------------------------------
            """ get the most recent pharmacy information """
            if registration['Pharmacy_Name__c'] != pharmacy_name[0]:
                if update_date >= pharmacy_name[1]:
                    pharmacy_name[0] = registration['Pharmacy_Name__c']
                    pharmacy_name[1] = registration['Date_Action_Recorded__c']

        refined_map[copay_card] = [document_list, pa_bucket, appeal_bucket, benefit_conversions, prescriber_bucket,
                                   insurance_bucket, pharmacy_name]
    # return the map
    return refined_map

<class 'dict'>
number of new cards in the feed: 392
[OrderedDict([('attributes', OrderedDict([('type', 'DTPC_Registration__c'), ('url', '/services/data/v42.0/sobjects/DTPC_Registration__c/a1H4P000007PnDHUA0')])), ('Hub_Patient_ID__c', 'P-0000241627'), ('PA_Submitted__c', None), ('PA_Submitted_Date__c', None), ('PA_Outcome__c', None), ('PA_Denial_Reason__c', None), ('PA_Expired__c', None), ('Appeal_Submitted__c', None), ('Appeal_Submitted_Date__c', None), ('Appeal_Outcome__c', None), ('Appeal_Denial_Reason__c', None), ('Benefit_Converted__c', None), ('Date_Action_Recorded__c', '2021-04-26'), ('DTPC_Prescriber_First_Name__c', 'CHRISTOPHER'), ('DTPC_Prescriber_Last_Name__c', 'D ARCY'), ('DTPC_Prescriber_Address__c', '45 WALLS ST STE 203B'), ('Prescriber_Address_2__c', None), ('DTPC_Prescriber_City__c', 'WESTERLY'), ('DTPC_Prescriber_State__c', 'RI'), ('Prescriber_Zip__c', '02891'), ('DTPC_Prescriber_Phone__c', '4013482180'), ('NPI__c', '1073560330'), ('DTPC_Copay_Card_Number__c', 'E461006